In [1]:
import json
from openai import OpenAI
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored 
import yfinance as yf
import pandas as pd
from matplotlib import pyplot as plt

GPT_MODEL = "gpt-3.5-turbo-0613"
client = OpenAI()

In [2]:
@retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, tools=None, tool_choice=None, model=GPT_MODEL):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
            tool_choice=tool_choice,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e


In [3]:
def get_stock_price(ticker, start_date, end_date):
    return yf.Ticker(ticker).history(start=start_date, end=end_date).Close


In [7]:
def create_portfolio(tickers, benchmark, start_date, end_date):
    portfolio = pd.DataFrame()
    factor = 1 / len(ticker)
    for ticker in tickers:
        temp_df = yf.Ticker(ticker).history(start=start_date, end=end_date)
        temp_df = temp_df.reset_index().set_index('Date')[['Close']]
        temp_df = temp_df.rename(columns={'Close': ticker})
        if portfolio.empty:
            portfolio = temp_df
            portfolio['port'] = temp_df[ticker] * factor
        else:
            portfolio = portfolio.merge(temp_df, left_index=True, right_index=True)
            portfolio['port'] += temp_df[ticker] * factor
        
        
    
    temp_benchmark = yf.Ticker(benchmark).history(start=start_date, end=end_date)
    portfolio = portfolio.merge(temp_benchmark, left_index=True, right_index=True)
    portfolio = portfolio[['port', benchmark]]
    return portfolio

# args = {'tickers': ['NVDA', 'AAPL'],
#  'benchmark': 'SP500',
#  'start_date': '2020-01-01',
#  'end_date': '2024-03-01'}

# create_portfolio(**args)

,NVDA,AAPL
Date,,
2020-01-02 00:00:00-05:00,59.741245,73.059441
2020-01-03 00:00:00-05:00,58.785023,72.349144
2020-01-06 00:00:00-05:00,59.031548,72.925644
2020-01-07 00:00:00-05:00,59.746212,72.582649
2020-01-08 00:00:00-05:00,59.858273,73.750252
...,...,...
2024-02-23 00:00:00-05:00,788.132996,182.520004
2024-02-26 00:00:00-05:00,790.882874,181.160004
2024-02-27 00:00:00-05:00,786.973083,182.630005


In [8]:
args = {'tickers': ['NVDA', 'AAPL'],
 'benchmark': 'SP500',
 'start_date': '2020-01-01',
 'end_date': '2024-03-01'}

df = create_portfolio(**args)

,NVDA,AAPL
Date,,
2020-01-02 00:00:00-05:00,1.000000,1.000000
2020-01-03 00:00:00-05:00,0.983994,0.990278
2020-01-06 00:00:00-05:00,0.988120,0.998169
2020-01-07 00:00:00-05:00,1.000083,0.993474
2020-01-08 00:00:00-05:00,1.001959,1.009455
...,...,...
2024-02-23 00:00:00-05:00,13.192443,2.498240
2024-02-26 00:00:00-05:00,13.238473,2.479625
2024-02-27 00:00:00-05:00,13.173028,2.499745


In [48]:
get_stock_price('AAPL', start_date='2023-01-01', end_date='2024-03-20')

Date
2023-01-03 00:00:00-05:00    124.216301
2023-01-04 00:00:00-05:00    125.497498
2023-01-05 00:00:00-05:00    124.166634
2023-01-06 00:00:00-05:00    128.735229
2023-01-09 00:00:00-05:00    129.261627
                                ...    
2024-03-13 00:00:00-04:00    171.130005
2024-03-14 00:00:00-04:00    173.000000
2024-03-15 00:00:00-04:00    172.619995
2024-03-18 00:00:00-04:00    173.720001
2024-03-19 00:00:00-04:00    176.080002
Name: Close, Length: 304, dtype: float64

In [53]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_stock_price",
            "description": "Get the close price of a stock given a ticker and start date and end date",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "The ticker of the stock. eg AAPL. Note: FB is now META",
                    },
                    "start_date": {
                        "type": "string",
                        "description": "start date in YYYY-MM-DD format",
                    },
                    "end_date": {
                        "type": "string",
                        "description": "start date in YYYY-MM-DD format",
                    },
                },
                "required": ["ticker", "start_date", 'end_date'],
            },
        }
    },
    {
        "type": "function",
        "function": {
            "name": "create_portfolio",
            "description": "Creates a portfolio by choosing a list of stock tickers and then compare it against a benchmark for a period with start date and end date",
            "parameters": {
                "type": "object",
                "properties": {
                    "tickers": {
                        "type": "array",
                        "items": {
                        "type": "integer"
                        },
                        "description": "List of tickers to be used portfolio construction. eg [AAPL, NVDA]. Note: FB is now META",
                    },
                    "benchmark": {
                        "type": "string",
                        "description": "The ticker of the benchmark. eg AAPL. Note: FB is now META",
                    },
                    "start_date": {
                        "type": "string",
                        "description": "start date in YYYY-MM-DD format",
                    },
                    "end_date": {
                        "type": "string",
                        "description": "start date in YYYY-MM-DD format",
                    },
                },
                "required": ["tickers", "benchmark", "start_date", 'end_date'],
            },
        }
    },
]

In [62]:
f_map = {
    'get_stock_price': get_stock_price,
    'create_portfolio': create_portfolio
}

In [56]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "create a portfolio of stocks using bitcoin and apple for the period 2020-01-01 to 2024-03-01. The benchmark is S&P 500"})
chat_response = chat_completion_request(
    messages, tools=tools
)
assistant_message = chat_response.choices[0].message
messages.append(assistant_message)
assistant_message


ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_7GThXdS9Umzv6ysEfIs3ZllD', function=Function(arguments='{\n  "tickers": ["BTC", "AAPL"],\n  "benchmark": "SP500",\n  "start_date": "2020-01-01",\n  "end_date": "2024-03-01"\n}', name='create_portfolio'), type='function')])

In [57]:
assistant_message.tool_calls[0].function

Function(arguments='{\n  "tickers": ["BTC", "AAPL"],\n  "benchmark": "SP500",\n  "start_date": "2020-01-01",\n  "end_date": "2024-03-01"\n}', name='create_portfolio')

In [58]:
json.loads(assistant_message.tool_calls[0].function.arguments)

{'tickers': ['BTC', 'AAPL'],
 'benchmark': 'SP500',
 'start_date': '2020-01-01',
 'end_date': '2024-03-01'}

In [63]:
f_name = f_map[assistant_message.tool_calls[0].function.name]
args = json.loads(assistant_message.tool_calls[0].function.arguments)

f_name(**args)

,Close


In [20]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "what is the weather going to be like in Glasgow, Scotland over the next x days"})
chat_response = chat_completion_request(
    messages, tools=tools
)
assistant_message = chat_response.choices[0].message
messages.append(assistant_message)
assistant_message


ChatCompletionMessage(content='Sure! Please provide the number of days for which you want to know the weather forecast.', role='assistant', function_call=None, tool_calls=None)

In [21]:
messages.append({"role": "user", "content": "5 days"})
chat_response = chat_completion_request(
    messages, tools=tools
)
chat_response.choices[0]

Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_At9XT132wmR6OCxAsSYvPoVm', function=Function(arguments='{\n  "location": "Glasgow, Scotland",\n  "format": "celsius",\n  "num_days": 5\n}', name='get_n_day_weather_forecast'), type='function')]))